# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

# Setting up connection to sqlite
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

import re

nltk.download(['punkt', 'wordnet', 'stopwords'])

# Make sure we can see all columns in our dataframes
pd.set_option('display.max_columns', 50)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\frank_tubbing\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\frank_tubbing\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\frank_tubbing\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
conn = sqlite3.connect('data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM messages', conn)
# I'm not gonna use the original message in the first place, id column has no added value. Might be that genre is usefull.
X = df[['message']]
# The data needs to be 1 dimensional for countvectorizer and tfidftransformer
X = X.message.tolist()
# Get all values that we are trying to predict
y = df.drop(columns=['id', 'message', 'original', 'genre'], axis=1)

In [3]:
# Inspect all target columns
y.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.0,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.00000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000,26180.000000
mean,0.773415,0.170474,0.004507,0.414095,0.079488,0.050115,0.027617,0.017991,0.032773,0.0,0.063751,0.111421,0.088197,0.015432,0.023033,0.011383,0.033384,0.045531,0.13136,0.064973,0.045760,0.050764,0.020321,0.006073,0.010810,0.004584,0.011803,0.043812,0.278189,0.082047,0.093201,0.010772,0.093659,0.020206,0.052445,0.193506
std,0.435359,0.376056,0.066986,0.492574,0.270504,0.218186,0.163875,0.132920,0.178046,0.0,0.244313,0.314659,0.283587,0.123264,0.150011,0.106083,0.179641,0.208470,0.33780,0.246483,0.208968,0.219519,0.141098,0.077696,0.103409,0.067549,0.108000,0.204681,0.448116,0.274442,0.290719,0.103228,0.291360,0.140708,0.222926,0.395054
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# Turns out that column child_alone has no value. So for this dataset it makes sense to remove this.
y.drop(columns=['child_alone'], axis=1, inplace=True)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # First remove punctation and lowercase all letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # lemmatize, stem and remove stop words
    tokens = [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.75)),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=500, min_samples_leaf=1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
for idx, column in enumerate(y_test.columns):
    print(column)
    print('_'*60)
    print(classification_report(y_test[column], y_pred[:,idx]))
    print("Confusion Matrix:\n", confusion_matrix(y_test[column], y_pred[:,idx]))
    print("\n")
    print("Accuracy: {0:.4f}".format((y_test[column] == y_pred[:,idx]).mean()))
    print("\n")

related
____________________________________________________________
              precision    recall  f1-score   support

           0       0.74      0.39      0.51      1532
           1       0.83      0.95      0.89      4963
           2       0.40      0.42      0.41        50

   micro avg       0.82      0.82      0.82      6545
   macro avg       0.65      0.59      0.60      6545
weighted avg       0.81      0.82      0.80      6545

Confusion Matrix:
 [[ 601  922    9]
 [ 212 4728   23]
 [   4   25   21]]


Accuracy: 0.8174


request
____________________________________________________________
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5436
           1       0.85      0.53      0.65      1109

   micro avg       0.90      0.90      0.90      6545
   macro avg       0.88      0.75      0.80      6545
weighted avg       0.90      0.90      0.89      6545

Confusion Matrix:
 [[5331  105]
 [ 523  586]]


Accuracy: 0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 [[3275  572]
 [ 768 1930]]


Accuracy: 0.7953


medical_help
____________________________________________________________
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6038
           1       0.61      0.05      0.09       507

   micro avg       0.92      0.92      0.92      6545
   macro avg       0.77      0.52      0.53      6545
weighted avg       0.90      0.92      0.89      6545

Confusion Matrix:
 [[6022   16]
 [ 482   25]]


Accuracy: 0.9239


medical_products
____________________________________________________________
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6226
           1       0.76      0.07      0.13       319

   micro avg       0.95      0.95      0.95      6545
   macro avg       0.86      0.53      0.55      6545
weighted avg       0.94      0.95      0.93      6545

Confusion Matrix:
 [[6219    7]
 [ 297   22]]


Accuracy: 0.9536


search_and_

Confusion Matrix:
 [[6464    0]
 [  81    0]]


Accuracy: 0.9876


other_infrastructure
____________________________________________________________
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6267
           1       0.00      0.00      0.00       278

   micro avg       0.96      0.96      0.96      6545
   macro avg       0.48      0.50      0.49      6545
weighted avg       0.92      0.96      0.94      6545

Confusion Matrix:
 [[6266    1]
 [ 278    0]]


Accuracy: 0.9574


weather_related
____________________________________________________________
              precision    recall  f1-score   support

           0       0.90      0.95      0.92      4718
           1       0.84      0.72      0.77      1827

   micro avg       0.88      0.88      0.88      6545
   macro avg       0.87      0.83      0.85      6545
weighted avg       0.88      0.88      0.88      6545

Confusion Matrix:
 [[4474  244]
 [ 517 1310]]


Accur

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenize at 0x000000000DE77950>,
          vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
              

In [27]:
parameters = {
    'vect__max_df': [0.5, 0.75, 1], #0.75
    'tfidf__sublinear_tf': [True, False], # False
    'clf__estimator__n_estimators' : [10, 100, 500], #500
    'clf__estimator__min_samples_leaf': [1, 2, 5] # 1
}    
# Volgende keer n_jobs=6, cv=5, verbose=2
cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_df': [0.5, 0.75, 1], 'tfidf__sublinear_tf': [True, False], 'clf__estimator__n_estimators': [10, 100, 500], 'clf__estimator__min_samples_leaf': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
y_pred2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
cv.best_estimator_.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.75, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenize at 0x000000000DE77950>,
          vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
              oob_score=False, random_state=None, verbose=0,
             

In [29]:
for idx, column in enumerate(y_test.columns):
    print(column)
    print('_'*60)
    print(classification_report(y_test[column], y_pred2[:,idx]))
    print("Confusion Matrix:\n", confusion_matrix(y_test[column], y_pred2[:,idx]))
    print("\n")
    print("Accuracy: {0:.4f}".format((y_test[column] == y_pred2[:,idx]).mean()))
    print("\n")

related
____________________________________________________________
              precision    recall  f1-score   support

           0       0.71      0.43      0.53      1488
           1       0.85      0.95      0.89      5006
           2       0.46      0.43      0.44        51

   micro avg       0.82      0.82      0.82      6545
   macro avg       0.67      0.60      0.62      6545
weighted avg       0.81      0.82      0.81      6545

Confusion Matrix:
 [[ 635  840   13]
 [ 254 4739   13]
 [   4   25   22]]


Accuracy: 0.8244


request
____________________________________________________________
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5425
           1       0.83      0.49      0.62      1120

   micro avg       0.90      0.90      0.90      6545
   macro avg       0.87      0.74      0.78      6545
weighted avg       0.89      0.90      0.88      6545

Confusion Matrix:
 [[5312  113]
 [ 567  553]]


Accuracy: 0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Confusion Matrix:
 [[3241  587]
 [ 769 1948]]


Accuracy: 0.7928


medical_help
____________________________________________________________
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6027
           1       0.65      0.08      0.14       518

   micro avg       0.92      0.92      0.92      6545
   macro avg       0.79      0.54      0.55      6545
weighted avg       0.90      0.92      0.90      6545

Confusion Matrix:
 [[6005   22]
 [ 477   41]]


Accuracy: 0.9238


medical_products
____________________________________________________________
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6197
           1       0.63      0.07      0.12       348

   micro avg       0.95      0.95      0.95      6545
   macro avg       0.79      0.53      0.55      6545
weighted avg       0.93      0.95      0.93      6545

Confusion Matrix:
 [[6183   14]
 [ 324   24]]


Accuracy: 0.

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6488
           1       0.00      0.00      0.00        57

   micro avg       0.99      0.99      0.99      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.98      0.99      0.99      6545

Confusion Matrix:
 [[6488    0]
 [  57    0]]


Accuracy: 0.9913


shops
____________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6516
           1       0.00      0.00      0.00        29

   micro avg       1.00      1.00      1.00      6545
   macro avg       0.50      0.50      0.50      6545
weighted avg       0.99      1.00      0.99      6545

Confusion Matrix:
 [[6516    0]
 [  29    0]]


Accuracy: 0.9956


aid_centers
____________________________________________________________
              precision    recall  f1-score   support

           0     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

In [13]:
for idx, column in enumerate(y_test.columns):
    print(column)
    print('_'*60)
    print(classification_report(y_test[column], y_pred[:,idx]))
    print("Confusion Matrix:\n", confusion_matrix(y_test[column], y_pred[:,idx]))
    print("\n")
    print("Accuracy: {0:.4f}".format((y_test[column] == y_pred[:,idx]).mean()))
    print("\n")

related
____________________________________________________________
              precision    recall  f1-score   support

           0       0.77      0.09      0.17      1510
           1       0.78      0.99      0.87      4986
           2       0.00      0.00      0.00        49

   micro avg       0.78      0.78      0.78      6545
   macro avg       0.52      0.36      0.35      6545
weighted avg       0.77      0.78      0.70      6545

Confusion Matrix:
 [[ 140 1370    0]
 [  38 4948    0]
 [   4   45    0]]


Accuracy: 0.7774


request
____________________________________________________________
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5429
           1       0.84      0.22      0.35      1116

   micro avg       0.86      0.86      0.86      6545
   macro avg       0.85      0.60      0.63      6545
weighted avg       0.86      0.86      0.82      6545

Confusion Matrix:
 [[5382   47]
 [ 872  244]]


Accuracy: 0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      5949
           1       0.33      0.00      0.00       596

   micro avg       0.91      0.91      0.91      6545
   macro avg       0.62      0.50      0.48      6545
weighted avg       0.86      0.91      0.87      6545

Confusion Matrix:
 [[5947    2]
 [ 595    1]]


Accuracy: 0.9088


clothing
____________________________________________________________
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6459
           1       0.00      0.00      0.00        86

   micro avg       0.99      0.99      0.99      6545
   macro avg       0.49      0.50      0.50      6545
weighted avg       0.97      0.99      0.98      6545

Confusion Matrix:
 [[6459    0]
 [  86    0]]


Accuracy: 0.9869


money
____________________________________________________________
              precision    recall  f1-score   support

           0       0

In [17]:
parameters = {
    'vect__max_df': [0.5, 0.75, 1], #0.75
    'tfidf__sublinear_tf': [True, False], # False
    'clf__estimator__alpha' : [0.1, 0.5, 1], #500
    'clf__estimator__fit_prior': [True, False] # 1
}    
# Volgende keer n_jobs=6, cv=5, verbose=2
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=2)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  38.9s
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  41.2s
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5 
[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  35.9s
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  40.3s
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  38.4s
[CV] clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.1, clf__estimator__fit_prior=True, tfidf_

[CV]  clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  35.1s
[CV] clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5 
[CV]  clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  39.9s
[CV] clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  38.4s
[CV] clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  36.5s
[CV] clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=0.5, clf__estimator__fit_prior=True, tfidf_

[CV]  clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  27.4s
[CV] clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5 
[CV]  clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.5, total=  27.1s
[CV] clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  26.5s
[CV] clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75, total=  26.7s
[CV] clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True, vect__max_df=0.75 
[CV]  clf__estimator__alpha=1, clf__estimator__fit_prior=True, tfidf__sublinear_tf=True

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 105.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'vect__max_df': [0.5, 0.75, 1], 'tfidf__sublinear_tf': [True, False], 'clf__estimator__alpha': [0.1, 0.5, 1], 'clf__estimator__fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [18]:
cv.best_estimator_.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.5, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenize at 0x000000000DBFFB70>,
          vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  MultiOutputClassifier(estimator=MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True),
             n_jobs=None))]

In [19]:
y_pred3 = cv.predict(X_test)

In [20]:
for idx, column in enumerate(y_test.columns):
    print(column)
    print('_'*60)
    print(classification_report(y_test[column], y_pred3[:,idx]))
    print("Confusion Matrix:\n", confusion_matrix(y_test[column], y_pred3[:,idx]))
    print("\n")
    print("Accuracy: {0:.4f}".format((y_test[column] == y_pred3[:,idx]).mean()))
    print("\n")

related
____________________________________________________________
              precision    recall  f1-score   support

           0       0.71      0.36      0.48      1510
           1       0.83      0.95      0.89      4986
           2       0.65      0.35      0.45        49

   micro avg       0.81      0.81      0.81      6545
   macro avg       0.73      0.55      0.61      6545
weighted avg       0.80      0.81      0.79      6545

Confusion Matrix:
 [[ 549  961    0]
 [ 219 4758    9]
 [   2   30   17]]


Accuracy: 0.8134


request
____________________________________________________________
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5429
           1       0.74      0.56      0.64      1116

   micro avg       0.89      0.89      0.89      6545
   macro avg       0.83      0.76      0.79      6545
weighted avg       0.88      0.89      0.89      6545

Confusion Matrix:
 [[5204  225]
 [ 488  628]]


Accuracy: 0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[6220   28]
 [ 264   33]]


Accuracy: 0.9554


search_and_rescue
____________________________________________________________
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6376
           1       0.40      0.01      0.02       169

   micro avg       0.97      0.97      0.97      6545
   macro avg       0.69      0.51      0.50      6545
weighted avg       0.96      0.97      0.96      6545

Confusion Matrix:
 [[6373    3]
 [ 167    2]]


Accuracy: 0.9740


security
____________________________________________________________
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6411
           1       0.00      0.00      0.00       134

   micro avg       0.98      0.98      0.98      6545
   macro avg       0.49      0.50      0.49      6545
weighted avg       0.96      0.98      0.97      6545

Confusion Matrix:
 [[6409    2]
 [ 134    0]]


Accuracy: 0.9792


military
______

### 9. Export your model as a pickle file

In [14]:
from joblib import dump, load
dump(pipeline, 'pipeline.pkl') 

['pipeline.pkl']

In [21]:
dump(cv, 'cv.pkl') 

['cv.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.